In [1]:
from libs.alibaba.alibaba import Alibaba
from libs.alibaba.inquiry import Inquiry
from libs.json import JSON

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotVisibleException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains

from datetime import datetime
from bs4 import BeautifulSoup
from pyquery import PyQuery as pq

import json
import requests
import time
import re
import math

import types
import traceback
import threading

import pendulum

import logging
from logging.handlers import TimedRotatingFileHandler

In [2]:
market = JSON.deserialize('.', 'storage', 'markets.json')['Eyelashes']
market

. storage markets.json
.\storage\markets.json was deserialized!


{'name': 'Eyelashes',
 'directory': 'E:\\glitter\\Eyelashes',
 'lid': 'eyelash@glittergroupcn.com',
 'lpwd': 'Wangzhen68970747',
 'accounts': [{'lid': 'eyelash1@glittergroupcn.com',
   'lpwd': 'Wangqingxia7696',
   'lname': 'Helen Wang',
   'disabled': False}],
 'homepage': 'https://glittereyelash.en.alibaba.com',
 'lname': 'Jeremy Wang'}

In [3]:
inquiry = Inquiry(market)
# inquiry.login()

E:\glitter\Eyelashes_config [] inquiry_tracking_ids.json
E:\glitter\Eyelashes_config\inquiry_tracking_ids.json was deserialized!


In [23]:
inquiry.login()

primary 打开网址：https://i.alibaba.com
primary 等待登陆页面加载 ... ...
primary 输入登录信息， 并登录
success 成功 登录 阿里巴巴 国际站！用户名：Jeremy


In [ ]:
def is_auto_reply_needed(self, enquiry):
    if enquiry['responsible_person'].lower() != self.lname.lower():
        return False
    eid = enquiry['id']
    if eid in self.tracking_ids && self.tracking_ids[eid].diff().in_hours()<=12:
        return True
    if len(enquiry['tags']) == 0 and enquiry['is_new']:
        return True
    
    return False

inquiry.is_auto_reply_needed = types.MethodType(is_auto_reply_needed, inquiry)

In [ ]:
def check(self):
    self.load_url()
    inquiries = self.get_inquiries()
    
    for enquiry in inquiries:
        if not self.is_auto_reply_needed(enquiry):
            continue
        self.reply(enquiry)
        
inquiry.check = types.MethodType(check, inquiry)

In [ ]:
def reply(self, enquiry):
    try:
        self.open_inquiry_in_new_tab(enquiry)
        conversation = self.get_conversation()

        messages = conversation['messages']
        buyer = conversation['buyer']

        last_buyer_message = ''
        is_replied = False
        reply_count = 0
        for msg in messages:
            if msg['position'] == 'left':
                last_buyer_message = msg
                is_replied = False
            elif msg['position'] == 'right' and 'Seller Assistant' not in msg['user']:
                is_replied = True
                reply_count += 1

        if reply_count == 0 and not is_replied:
            self.reply_to_new_inquiry(last_buyer_message)
            if enquiry['id'] not in self.tracking_ids:
                self.tracking_ids[enquiry['id']] = pendulum.now()
                self.save_tracking_ids()
        elif reply_count != 0 and not is_replied:
            self.reply_to_message(last_buyer_message)
        else:
            self.notify("wanning", '该询盘 [' + enquiry['id'] + '] 已经被回复，取消 介入')
    except Exception as e:
        self.notify("danger", '回复 询盘 [' + enquiry['id'] + '] 时 发生错误! ' + str(e))
        traceback.print_exc()
    finally:
        self.close_inquiry_and_switch_back()

inquiry.reply = types.MethodType(reply, inquiry)

In [ ]:
def reply_to_new_inquiry(self, message):
    message = 'product catalog'
    self.notify('primary', 'Inquiry['+enquiry['id']+'] is a new and will be replid with message: ' + message)
    self.send_message(message)
inquiry.reply_to_new_inquiry = types.MethodType(reply_to_new_inquiry, inquiry)

In [ ]:
def reply_to_message(self, message):
    message = 'product catalog'
    self.notify('primary', 'Inquiry['+enquiry['id']+'] is not new and will be replid with message: ' + message)
    self.send_message(message)
inquiry.reply_to_message = types.MethodType(reply_to_message, inquiry)

In [51]:
def send_message(self, message, attachment=None):
    chat_form = self.browser.find_element_by_css_selector('form.reply-wrapper')
    chat_form.click()
    
    WebDriverWait(self.browser, 15).until(EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR, '#inquiry-content_ifr')))
    body = self.browser.find_element_by_tag_name('body')
    body.click()
    body.send_keys(message)
    self.browser.switch_to.default_content()
    btn_send = chat_form.find_element_by_css_selector('button.send')
#     btn_send.click()
inquiry.send_message = types.MethodType(send_message, inquiry)

In [ ]:
inquiry.load_url()
inquiries = inquiry.get_inquiries()

In [39]:
    chat_form = inquiry.browser.find_element_by_css_selector('form.reply-wrapper')
    chat_form.click()


In [28]:

btn_send = chat_form.find_element_by_css_selector('button.send')

In [50]:
# WebDriverWait(inquiry.browser, 15).until(EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR, '#inquiry-content_ifr')))
# inquiry.browser.find_
inquiry.browser.switch_to.default_content()

In [36]:
iframe = inquiry.browser.find_element_by_css_selector('#inquiry-content_ifr')

In [37]:
iframe.location

{'x': 720, 'y': 718}

In [38]:
iframe.size

{'height': 150, 'width': 625}

In [ ]:
inquiry.check()

In [40]:
WebDriverWait(inquiry.browser, 15).until(EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR, '#inquiry-content_ifr')))

True

In [41]:
inquiry.browser.find_element_by_tag_name('body').click()

In [ ]:
inquiry.browser.switch_to.default_content()

In [ ]:
inquiry.close_inquiry_and_switch_back()

In [ ]:
buyer

In [ ]:
messages

In [ ]:
inquiries[0]

In [ ]:
enquiry = Inquiry(market)
enquiry.login()

In [ ]:
date_str = pendulum.now().to_atom_string()
print(date_str)
date = pendulum.parse(date_str)
print(date)

In [15]:
inquiry.tracking_ids['100693358415'] = pendulum.now()
# self.save_tracking_ids()

In [19]:
inquiry.tracking_ids = {}

In [17]:
inquiry.save_tracking_ids()

E:\glitter\Eyelashes_config\inquiry_tracking_ids.json was serialized!


In [16]:
inquiry.tracking_ids

{'100693358415': DateTime(2018, 7, 10, 16, 5, 3, 717237, tzinfo=Timezone('Asia/Shanghai'))}

In [21]:
inquiry.load_tracking_ids()

E:\glitter\Eyelashes_config [] inquiry_tracking_ids.json
E:\glitter\Eyelashes_config\inquiry_tracking_ids.json was deserialized!


In [ ]:
date.diff().in_hours()<=12

In [22]:
inquiry.tracking_ids

{'100693358415': DateTime(2018, 7, 10, 16, 5, 3, tzinfo=Timezone('+08:00'))}